# Aspect-Aware Sentiment Modeling for Educational Feedback
Formalizes the research-exchange idea with runnable baselines, advanced models, prompting hooks, and explainability for joint aspect+sentiment.

## 1. Setup
This notebook provides executable code blocks from data loading through training, evaluation, explainability, and CLI entry points. Install extras (`transformers`, `sentence-transformers`, `shap`, `lime`, `openai`) if they are not already available in your environment.

In [ ]:
import os
import json
import random
from dataclasses import dataclass
from pathlib import Path
from typing import List, Dict, Optional

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

import shap
from lime.lime_text import LimeTextExplainer

# Optional heavy deps (guarded imports)
try:
    from sentence_transformers import SentenceTransformer
except Exception:
    SentenceTransformer = None

try:
    from transformers import (
        AutoTokenizer,
        AutoModelForSequenceClassification,
        Trainer,
        TrainingArguments,
        DataCollatorWithPadding,
    )
    import datasets
    from torch import nn
    import torch
except Exception:
    AutoTokenizer = AutoModelForSequenceClassification = Trainer = TrainingArguments = DataCollatorWithPadding = None
    datasets = nn = torch = None

In [ ]:
@dataclass
class Config:
    data_path: Path = Path("data_feedback.xlsx")
    text_col: str = "comments"
    aspect_col: str = "teacher/course"
    label_col: str = "sentiment"
    random_state: int = 42

In [ ]:
def load_data(cfg: Config) -> pd.DataFrame:
    if cfg.data_path.exists():
        df = pd.read_excel(cfg.data_path)
    else:
        df = pd.DataFrame(
            {
                "teacher/course": ["teacher", "course"],
                "comments": ["great teacher", "great course"],
                "sentiment": ["positive", "positive"],
            }
        )
    df = df.rename(columns={cfg.text_col: "text", cfg.aspect_col: "aspect_tag", cfg.label_col: "label"})
    df = df.dropna(subset=["text", "aspect_tag", "label"]).reset_index(drop=True)
    return df

## 2. Data audit and splits
Includes aspect-specific splits to enable cross-aspect transfer evaluation.

In [ ]:
def train_val_split(df: pd.DataFrame, cfg: Config):
    strat = df["label"] if df["label"].nunique() > 1 else None
    train_df, val_df = train_test_split(df, test_size=0.25, random_state=cfg.random_state, stratify=strat)
    return train_df.reset_index(drop=True), val_df.reset_index(drop=True)

cfg = Config()
df = load_data(cfg)
train_df, val_df = train_val_split(df, cfg)
train_df.head(), val_df.head()

## 3. Preprocessing helpers and evaluation utilities

In [ ]:
def prepend_aspect(texts: List[str], aspects: List[str]):
    return [f"[ASPECT={a}] {t}" for t, a in zip(texts, aspects)]


def evaluate(model, X_val, y_val, target_names=None, label="eval"):
    preds = model.predict(X_val)
    report = classification_report(y_val, preds, target_names=target_names, zero_division=0)
    cm = confusion_matrix(y_val, preds)
    print(f"
==== {label} report ====")
    print(report)
    print("Confusion matrix:
", cm)
    return report, cm

## 4. N-gram TF–IDF baselines (word + character)
Includes aspect prompts to test aspect-aware conditioning.

In [ ]:
def run_tfidf_baseline(train_df, val_df, use_aspect_prompt=False, analyzer="word", ngram_range=(1, 2)):
    X_train = train_df["text"] if not use_aspect_prompt else prepend_aspect(train_df["text"].tolist(), train_df["aspect_tag"].tolist())
    X_val = val_df["text"] if not use_aspect_prompt else prepend_aspect(val_df["text"].tolist(), val_df["aspect_tag"].tolist())
    y_train, y_val = train_df["label"], val_df["label"]

    pipe = Pipeline(
        [
            ("tfidf", TfidfVectorizer(analyzer=analyzer, ngram_range=ngram_range, min_df=1)),
            ("clf", LogisticRegression(max_iter=200, class_weight="balanced")),
        ]
    )
    pipe.fit(X_train, y_train)
    return pipe, evaluate(pipe, X_val, y_val, label=f"tfidf-{analyzer}-aspect={use_aspect_prompt}")

## 5. Sentence-embedding classifier (SBERT + LogisticRegression)
Uses aspect prompts to provide aspect-aware context.

In [ ]:
def run_sbert_classifier(train_df, val_df, model_name: str = "all-MiniLM-L6-v2", use_aspect_prompt=True):
    if SentenceTransformer is None:
        raise ImportError("sentence-transformers not installed")

    model = SentenceTransformer(model_name)
    X_train = train_df["text"] if not use_aspect_prompt else prepend_aspect(train_df["text"].tolist(), train_df["aspect_tag"].tolist())
    X_val = val_df["text"] if not use_aspect_prompt else prepend_aspect(val_df["text"].tolist(), val_df["aspect_tag"].tolist())
    y_train, y_val = train_df["label"], val_df["label"]

    train_emb = model.encode(list(X_train), batch_size=16, show_progress_bar=True)
    val_emb = model.encode(list(X_val), batch_size=16, show_progress_bar=True)

    clf = LogisticRegression(max_iter=200, class_weight="balanced")
    clf.fit(train_emb, y_train)
    evaluate(clf, val_emb, y_val, label=f"sbert-aspect={use_aspect_prompt}")
    return clf, model

## 6. Transformer fine-tuning (aspect-prompted)
Lightweight Trainer setup for reproducibility.

In [ ]:
def prepare_hf_dataset(train_df, val_df, tokenizer):
    def tokenize(batch):
        texts = prepend_aspect(batch["text"], batch["aspect_tag"])
        return tokenizer(texts, truncation=True, max_length=256)

    train_ds = datasets.Dataset.from_pandas(train_df)
    val_ds = datasets.Dataset.from_pandas(val_df)
    return train_ds.map(tokenize, batched=True), val_ds.map(tokenize, batched=True)


def run_transformer(train_df, val_df, model_name="distilbert-base-uncased", num_epochs=3):
    if AutoTokenizer is None:
        raise ImportError("transformers not installed")

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=train_df["label"].nunique())

    train_ds, val_ds = prepare_hf_dataset(train_df, val_df, tokenizer)
    collator = DataCollatorWithPadding(tokenizer)

    args = TrainingArguments(
        output_dir="./outputs",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        logging_steps=50,
    )

    def compute_metrics(eval_pred):
        preds, labels = eval_pred
        pred_labels = preds.argmax(axis=1)
        report = classification_report(labels, pred_labels, output_dict=True, zero_division=0)
        return {"macro_f1": report["macro avg"]["f1-score"]}

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        data_collator=collator,
        compute_metrics=compute_metrics,
    )
    trainer.train()
    return trainer

## 7. Multi-task (aspect + sentiment) head for joint learning
Shared encoder with dual classification heads to exploit aspect cues when predicting sentiment.

In [ ]:
class MultiTaskHead(nn.Module):
    def __init__(self, hidden_size, num_sentiment, num_aspect):
        super().__init__()
        self.dropout = nn.Dropout(0.1)
        self.sentiment_classifier = nn.Linear(hidden_size, num_sentiment)
        self.aspect_classifier = nn.Linear(hidden_size, num_aspect)

    def forward(self, features, **kwargs):
        x = self.dropout(features)
        return {
            "sentiment": self.sentiment_classifier(x),
            "aspect": self.aspect_classifier(x),
        }

## 8. Cross-aspect robustness (train on teacher, test on course and vice versa)

In [ ]:
def cross_aspect_eval(df, model_fn):
    teacher_df = df[df["aspect_tag"] == "teacher"].reset_index(drop=True)
    course_df = df[df["aspect_tag"] == "course"].reset_index(drop=True)
    results = {}
    for train_df, test_df, tag in [
        (teacher_df, course_df, "train_teacher_test_course"),
        (course_df, teacher_df, "train_course_test_teacher"),
    ]:
        model, _ = model_fn(train_df, test_df)
        results[tag] = model
    return results

## 9. Lightweight data augmentation for robustness
Simple synonym/word-drop augmentations; plug into any experiment.

In [ ]:
def random_drop(text, p=0.15):
    words = text.split()
    keep = [w for w in words if random.random() > p]
    return " ".join(keep) if keep else text


def augment_dataframe(df, times=1):
    rows = []
    for _ in range(times):
        for _, row in df.iterrows():
            rows.append({
                "text": random_drop(row["text"]),
                "aspect_tag": row["aspect_tag"],
                "label": row["label"],
            })
    return pd.concat([df, pd.DataFrame(rows)], ignore_index=True)

## 10. Zero/low-shot prompting baseline (LLM)
Uses explicit schema and aspect cues; keep API keys in environment variables.

In [ ]:
def prompt_sentiment(texts: List[str], aspects: List[str], model_name: str = "gpt-4o-mini"):
    import openai

    client = openai.OpenAI()
    outputs = []
    for t, a in zip(texts, aspects):
        res = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": "Classify sentiment as positive, neutral, or negative and explain briefly."},
                {"role": "user", "content": f"Aspect: {a}. Comment: {t}"},
            ],
            temperature=0,
        )
        outputs.append(res.choices[0].message["content"])
    return outputs

## 11. Error analysis and explainability reports
Combine SHAP/LIME outputs with per-length/per-aspect slices.

In [ ]:
def error_table(model, val_df, use_aspect_prompt=True):
    X_val = val_df["text"] if not use_aspect_prompt else prepend_aspect(val_df["text"].tolist(), val_df["aspect_tag"].tolist())
    y_true = val_df["label"].tolist()
    preds = model.predict(X_val)
    df_err = val_df.copy()
    df_err["pred"] = preds
    df_err["correct"] = df_err["pred"] == df_err["label"]
    return df_err.sort_values("correct")


def explain_with_shap(model, X_samples: List[str], class_names: List[str]):
    explainer = shap.Explainer(model.predict_proba, masker=shap.maskers.Text())
    shap_values = explainer(X_samples)
    shap.plots.text(shap_values, display=False)
    return shap_values


def explain_with_lime(model, X_samples: List[str], class_names: List[str]):
    explainer = LimeTextExplainer(class_names=class_names)
    explanations = [explainer.explain_instance(x, model.predict_proba, num_features=8) for x in X_samples]
    return explanations

## 12. CLI entry points
Run from terminal: `python -m sentiment_analysis --model tfidf` etc.

In [ ]:
def main_cli():
    import argparse

    parser = argparse.ArgumentParser(description="Aspect-aware sentiment experiments")
    parser.add_argument("--model", choices=["tfidf", "char", "sbert", "transformer"], default="tfidf")
    args = parser.parse_args()

    cfg = Config()
    df = load_data(cfg)
    train_df, val_df = train_val_split(df, cfg)

    if args.model == "tfidf":
        run_tfidf_baseline(train_df, val_df, use_aspect_prompt=False)
    elif args.model == "char":
        run_tfidf_baseline(train_df, val_df, use_aspect_prompt=True, analyzer="char", ngram_range=(3, 5))
    elif args.model == "sbert":
        run_sbert_classifier(train_df, val_df)
    elif args.model == "transformer":
        run_transformer(train_df, val_df)


if __name__ == "__main__":
    main_cli()